In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcsetup.all_backends
import pickle

import pyExSi

import LadiskDAQ
import os

%matplotlib qt

In [2]:
acq1 = LadiskDAQ.NIAcquisition("TestInputTask", acquisition_name="NI_task")

acq2 = LadiskDAQ.SerialAcquisition(port="COM3", baudrate=250000, timeout=5,
                                  byte_sequence=(("int16", 1), ("int16", 1)),
                                  start_bytes=b"\xfa\xfb",
                                  end_bytes=b"\n",
                                  pretest_time= None,
                                  sample_rate=500.,
                                  acquisition_name="Arduino_1"
                                  )

acq3 = LadiskDAQ.SerialAcquisition(port="COM8", baudrate=250000, timeout=5,
                                  byte_sequence=(("int16", 1), ("int16", 1)),
                                  start_bytes=b"\xfa\xfb", 
                                  end_bytes=b"\n",
                                  pretest_time= None,
                                  sample_rate=200,
                                  acquisition_name="Arduino_2"
                                  )


In [3]:
acq1.sample_rate, acq2.sample_rate, acq3.sample_rate

(25600.0, 500.0, 200)

In [4]:
# create DAQ object:
ldaq = LadiskDAQ.Core(acquisitions=[acq3, acq1, acq2])
#ldaq.set_global_trigger(
#     trigger_source = "Arduino_1", 
#     trigger_channel = 1, 
#     trigger_level = 10000,
#)

# run v trigger, 
# set_trigger()
# ta trigger naštima acq trigger in ne naredi svoj trigger.

In [8]:
ldaq.run(run_time=120., run_name="RunTest", save_interval=1., root='C:/LADISK/packages/test_meas', verbose=1)

	Waiting for trigger...acq
acq
acq
check
per_save


In [6]:
measurement_dict = ldaq.get_measurement_dict(N_seconds=None)

root = r"C:\LADISK\packages\test_meas"
file = os.listdir(root)[-1]
measurement_dict_saved = LadiskDAQ.load_measurement(file, root)

In [7]:
fig, axs = plt.subplots(3, sharex=True)
for idx, source in enumerate(list(measurement_dict.keys())):
    meas = measurement_dict[source]
    meas_saved = measurement_dict_saved[source]
    axs[idx].plot( meas["time"], meas["data"] )
    axs[idx].plot( meas_saved["time"], meas_saved["data"] )
